# 数据预处理
* 数据来源：北京到上海航线数据，../datasets/route_price_prediction_beijing_shanghai.csv
    * 原始特征：['log_date', 'dep_date', 'dep_city', 'arr_city', 'pre_day',
         'search_num_30days', 'search_num_15days','search_num_7days',
         'booking_num_30days', 'booking_num_15days', 'booking_num_7days',
         'order_num_30days', 'order_num_15days', 'order_num_7days',
         'search_num_dep_city', 'search_num_arr_city', 'minprice']
* 代码内容：
    * 处理数据格式
    * 用2017年的数据作为训练集，增加2016年数据作为17年的历史同期特征
    * 生成模型可用的csv

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot
import datetime
import time
from sklearn.preprocessing import MinMaxScaler

## 1. 北京到上海航线数据导入和总览

In [3]:
df_bs = pd.read_csv('../datasets/rpp_20180510.csv',sep='\t')

In [4]:
df_bs.shape

(14660711, 17)

In [5]:
df_bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14660711 entries, 0 to 14660710
Data columns (total 17 columns):
dep_city                      object
arr_city                      object
flightno                      object
search_date                   object
dep_date                      object
pre_day                       int64
search_num_15days             int64
booking_num_15days            int64
total_order_num               int64
total_ticket_price            float64
search_num                    int64
booking_num                   int64
order_num                     int64
ticket_price                  float64
search_dep_city_search_num    int64
search_arr_city_search_num    int64
minprice                      float64
dtypes: float64(3), int64(9), object(5)
memory usage: 1.9+ GB


In [6]:
df_bs.columns

Index(['dep_city', 'arr_city', 'flightno', 'search_date', 'dep_date',
       'pre_day', 'search_num_15days', 'booking_num_15days', 'total_order_num',
       'total_ticket_price', 'search_num', 'booking_num', 'order_num',
       'ticket_price', 'search_dep_city_search_num',
       'search_arr_city_search_num', 'minprice'],
      dtype='object')

## 2. 数据处理

### 2.1 原始数据集处理
* 重命名
* 格式转换
* 指定起飞日期范围并排序
* 去除超出60天数据

In [8]:
# 重命名：接手代码，减少修改，快速出结果
df_bs = df_bs.rename(index=str,columns={'search_date':"orderDate",'dep_date':"depDate",'minprice':'lowestPrice'})  
# 将object转化为日期格式: errors = 'coerce'，无效解析置为NaT
df_bs['orderDate'] = pd.to_datetime(df_bs['orderDate'],errors='coerce')
df_bs['depDate']   = pd.to_datetime(df_bs['depDate'],errors='coerce')
# 指定起飞日期范围
df_bs = df_bs[df_bs['depDate']>='2016-03-01']
df_bs = df_bs[df_bs['depDate']<='2018-03-01']
# 按照起飞日期排序
df_bs.sort_values(by=['depDate','orderDate'],inplace=True)
# 去除搜索日期超出起飞日期前60天的数据
df_bs = df_bs[df_bs.orderDate >= df_bs.depDate - datetime.timedelta(days = 59)]

In [9]:
df_bs.shape

(9372025, 17)

In [118]:
df_bs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43844 entries, 34945 to 45639
Data columns (total 17 columns):
orderDate              43844 non-null datetime64[ns]
depDate                43844 non-null datetime64[ns]
dep_city               43844 non-null object
arr_city               43844 non-null object
pre_day                43844 non-null int64
search_num_30days      43844 non-null int64
search_num_15days      43844 non-null int64
search_num_7days       43844 non-null int64
booking_num_30days     43844 non-null int64
booking_num_15days     43844 non-null int64
booking_num_7days      43844 non-null int64
order_num_30days       43844 non-null int64
order_num_15days       43844 non-null int64
order_num_7days        43844 non-null int64
search_num_dep_city    43844 non-null int64
search_num_arr_city    43844 non-null int64
lowestPrice            43844 non-null int64
dtypes: datetime64[ns](2), int64(13), object(2)
memory usage: 6.0+ MB


### 2.2 缺失数据填充

In [192]:
date_range = pd.date_range('2016-03-01','2018-3-1') # 起飞时间范围
features = ['search_num_30days', 'search_num_15days', 'search_num_7days',
       'booking_num_30days', 'booking_num_15days', 'booking_num_7days',
       'order_num_30days', 'order_num_15days', 'order_num_7days',
       'search_num_dep_city', 'search_num_arr_city', 'lowestPrice']
fill_data,del_data = data_missing_process(df_bs,date_range,features)

缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-01 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-02 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-03 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-04 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-05 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-06 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-07 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-08 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-09 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-10 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-03-11 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
I

缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-06-28 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-06-29 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-06-30 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-01 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-02 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-03 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-04 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-05 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-06 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-07 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-07-08 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
I

缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-18 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-19 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-20 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-21 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-22 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-23 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-24 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-25 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-26 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-27 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2016-10-28 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
I

缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-01-29 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-01-30 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-01-31 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-01 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-02 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-03 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-04 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-05 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-06 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-07 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-02-08 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
I

缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-05-26 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-05-27 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-05-28 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-05-29 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-05-30 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-05-31 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-06-01 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-06-02 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-06-03 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-06-04 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-06-05 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
I

缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-21 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-22 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-23 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-24 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-25 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-26 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-27 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-28 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-29 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-09-30 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-10-01 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
I

缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-12-30 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2017-12-31 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-01 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-02 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-03 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-04 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-05 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-06 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-07 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-08 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
Index: []
2018-01-09 00:00:00缺失填充完成！缺失数据大小为0
缺失数据为Empty DataFrame
Columns: [orderDate]
I

In [193]:
# 填充pre_day
fill_data['pre_day'] = fill_data['depDate']-fill_data['orderDate']
fill_data['pre_day'] = (fill_data['pre_day'] / np.timedelta64(1, 'D')).astype(int)

In [140]:
fill_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 15 columns):
depDate                16 non-null object
orderDate              16 non-null object
search_num_30days      16 non-null object
search_num_15days      16 non-null object
search_num_7days       16 non-null object
booking_num_30days     16 non-null object
booking_num_15days     16 non-null object
booking_num_7days      16 non-null object
order_num_30days       16 non-null object
order_num_15days       16 non-null object
order_num_7days        16 non-null object
search_num_dep_city    16 non-null object
search_num_arr_city    16 non-null object
lowestPrice            16 non-null object
pre_day                16 non-null int64
dtypes: int64(1), object(14)
memory usage: 2.6+ KB


In [141]:
fill_data.head()

,depDate,orderDate,search_num_30days,search_num_15days,search_num_7days,booking_num_30days,booking_num_15days,booking_num_7days,order_num_30days,order_num_15days,order_num_7days,search_num_dep_city,search_num_arr_city,lowestPrice,pre_day
0,2016-08-05 00:00:00,2016-06-18 00:00:00,1.28694e+06,663918,309864,183651,92413,40795,12225,5949.5,2614,354814,340528,535,48
1,2016-08-18 00:00:00,2016-06-25 00:00:00,1.30482e+06,656954,304158,184860,88917.5,41979,12034.5,5654.5,2700.5,370418,347738,545,54
2,2016-10-30 00:00:00,2016-09-10 00:00:00,1.46781e+06,725178,341502,339179,180088,87989,13381.5,6860,3154.5,428104,379321,540,50
3,2016-11-08 00:00:00,2016-09-11 00:00:00,1.46133e+06,722101,336138,342530,181606,88710,13384,6837.5,3109.5,414764,370918,450,58
4,2016-11-09 00:00:00,2016-09-12 00:00:00,1.45646e+06,718540,331816,345874,184755,89844.5,13399.5,6822.5,3091.5,433741,384223,480,58


### 2.3 合并填充数据

In [142]:
# 合并前的数据
df_bs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43844 entries, 19221 to 129357
Data columns (total 17 columns):
orderDate              43844 non-null datetime64[ns]
depDate                43844 non-null datetime64[ns]
dep_city               43844 non-null object
arr_city               43844 non-null object
pre_day                43844 non-null int64
search_num_30days      43844 non-null int64
search_num_15days      43844 non-null int64
search_num_7days       43844 non-null int64
booking_num_30days     43844 non-null int64
booking_num_15days     43844 non-null int64
booking_num_7days      43844 non-null int64
order_num_30days       43844 non-null int64
order_num_15days       43844 non-null int64
order_num_7days        43844 non-null int64
search_num_dep_city    43844 non-null int64
search_num_arr_city    43844 non-null int64
lowestPrice            43844 non-null int64
dtypes: datetime64[ns](2), int64(13), object(2)
memory usage: 7.3+ MB


In [194]:
# 与原数据合并
fill_data['orderDate'] = pd.to_datetime(fill_data['orderDate'],errors='coerce')
fill_data['depDate']   = pd.to_datetime(fill_data['depDate'],errors='coerce')
df_bs = df_bs.append(fill_data,ignore_index=True)
print(df_bs.shape)

(43860, 17)


In [144]:
# 合并后的数据
df_bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43860 entries, 0 to 43859
Data columns (total 17 columns):
arr_city               43844 non-null object
booking_num_15days     43860 non-null object
booking_num_30days     43860 non-null object
booking_num_7days      43860 non-null object
depDate                43860 non-null datetime64[ns]
dep_city               43844 non-null object
lowestPrice            43860 non-null object
orderDate              43860 non-null datetime64[ns]
order_num_15days       43860 non-null object
order_num_30days       43860 non-null object
order_num_7days        43860 non-null object
pre_day                43860 non-null int64
search_num_15days      43860 non-null object
search_num_30days      43860 non-null object
search_num_7days       43860 non-null object
search_num_arr_city    43860 non-null object
search_num_dep_city    43860 non-null object
dtypes: datetime64[ns](2), int64(1), object(14)
memory usage: 5.7+ MB


In [200]:
df_bs.head()

,dept_order_date,arr_city,booking_num_15days,booking_num_30days,booking_num_7days,dep_city,order_num_15days,order_num_30days,order_num_7days,pre_day,search_num_15days,search_num_30days,search_num_7days,search_num_arr_city,search_num_dep_city,lowestPrice
0,"2016-03-01,2016-01-02",上海,25698,25698,25698,北京,1087,1087,1087,59,86877,86877,86877,385303,475045,345.0
1,"2016-03-01,2016-01-03",上海,37889,37889,37889,北京,1549,1549,1549,58,130230,130230,130230,420547,529044,304.0
2,"2016-03-01,2016-01-04",上海,51337,51337,51337,北京,2245,2245,2245,57,182370,182370,182370,476852,623209,295.0
3,"2016-03-01,2016-01-05",上海,63982,63982,63982,北京,2958,2958,2958,56,257722,257722,257722,500830,656866,295.0
4,"2016-03-01,2016-01-06",上海,79021,79021,79021,北京,3616,3616,3616,55,334054,334054,334054,517206,638857,266.0


## 3.保存数据到datasets

In [203]:
# beijing_to_kunming_csv = '../datasets/bk_feature_process_csv'
# chengdu_to_shenzhen_csv = '../datasets/cs_feature_process_csv'
# chengdu_to_guangzhou_csv = '../datasets/cg_feature_process_csv'
# kunming_to_shanghai_csv = '../datasets/ks_feature_process_csv'
df_bs.to_csv('../datasets/bs_feature_process.csv', sep=',',header=True,index=False,encoding="utf_8_sig")
# df_bk.to_csv(beijing_to_kunming_csv, sep=',',header=True,index=False,encoding="utf_8_sig")
# df_cs.to_csv(chengdu_to_shenzhen_csv, sep=',',header=True,index=False,encoding="utf_8_sig")
# df_cg.to_csv(chengdu_to_guangzhou_csv, sep=',',header=True,index=False,encoding="utf_8_sig")
# df_ks.to_csv(kunming_to_shanghai_csv, sep=',',header=True,index=False,encoding="utf_8_sig")
print('北京-上海的数据保存至../datasets/bs_feature_process.csv')
# print('北京-昆明的数据保存至',beijing_to_kunming_csv)
# print('成都-深圳的数据保存至',chengdu_to_shenzhen_csv)
# print('成都-广州的数据保存至',chengdu_to_guangzhou_csv)
# print('昆明-上海的数据保存至',kunming_to_shanghai_csv)

北京-上海的数据保存至../datasets/bs_feature_process.csv


## 以下是上述代码用到的函数

### 1. 特征工程

In [64]:
def feature_engineering(df):
    # 2.构造出发时间所属月份
    df['dep_month'] = df.depDate.apply(lambda x : x.month)
    # 3.构造出发时间所属星期
    df['dep_week']  = df.depDate.apply(lambda x : x.weekday()+1)
    # 4.构造出发时间所属季节
    df['dep_season'] = df.dep_month.apply(lambda x : get_takeoff_season(x))
    # 5.构造出发时间是否为周末
    df['dep_is_weekend'] = df.dep_week.apply(lambda x : '0' if x <= 5 else '1')
    # 6.构造起飞时间是否为春节
    df['dep_is_springfestival'] = df.dep_month.apply(lambda x : '1' if x <= 2 else '0')
    # 7.构造起飞时间是否为劳动节
    df['dep_is_labourday'] = df.depDate.apply(lambda x : is_labourday(x))
    # 8.构造起飞时间是否为国庆节
    df['dep_is_nationalday'] = df.depDate.apply(lambda x : is_nationalday(x))
    # 9.构造起飞时间是否为元旦节
    df['dep_is_newyearday'] = df.depDate.apply(lambda x : is_newyearday(x))
    # 10.构造起飞时间是否为暑假
    df['dep_is_summervacation'] = df.dep_month.apply(lambda x : '1' if x >= 7 and x <= 9 else '0')
    # 11.构造起飞时间为月初、月中或者月末
    df['dep_month_segment'] = df.depDate.apply(lambda x : get_month_segment(x))
    # 12.构造起飞时间是否为节假日
    df['dep_is_festival'] = df.depDate.apply(lambda x : is_festival(x))
    # 13.构造订票时间所属星期
    df['order_week']  = df.orderDate.apply(lambda x : x.weekday()+1)
    # 14.构造订票时间是否为周末
    df['order_is_weekend'] = df.order_week.apply(lambda x : '0' if x <= 5 else '1')
    # 15.构造订票时间是否为节假日
    df['order_is_festival'] = df.orderDate.apply(lambda x : is_festival(x))
    # 16.构造起飞城市当天搜索量排名
    df['dep_city_rank'] = df.depCityCount.rank(method='average')
    # 17.构造目的城市当天搜索量排名
    df['arr_city_rank'] = df.arrCityCount.rank(method='average')

#### 1.1 获取起飞季节字段

In [65]:
def get_takeoff_season(m):
    if m <= 3:
        season = '1'  # 春天
    elif m <= 6:
        season = '2'  # 夏天
    elif m <= 9:
        season = '3'  # 秋天
    elif m <= 12:
        season = '4'  # 冬天
    return season

#### 1.2 获取起飞时间是否为劳动节字段

In [66]:
def is_labourday(d):
    if d.month == 5 and d.day >= 1 and d.day <= 3:
        IsLabourDay = '1'  # 劳动节
    elif d.month == 4 and d.is_month_end == True:  # 劳动节前一天
        IsLabourDay = '1'  # 劳动节
    elif d.month == 5 and d.day == 4:  # 劳动节后一天
        IsLabourDay = '1'  # 劳动节
    else:
        IsLabourDay = '0'  # 非劳动节
    return IsLabourDay

#### 1.3 获取起飞时间是否为国庆节字段

In [67]:
def is_nationalday(d):
    if d.month == 10 and d.day >= 1 and d.day <= 7:
        IsNationalDay = '1'  # 国庆节
    elif d.month == 9 and d.is_month_end == True:  # 国庆节前一天
        IsNationalDay = '1'  # 国庆节
    elif d.month == 10 and d.day == 8:  # 国庆节后一天
        IsNationalDay = '1'  # 国庆节
    else:
        IsNationalDay = '0'  # 非国庆节
    return IsNationalDay

#### 1.4 获取起飞时间是否为元旦节

In [68]:
def is_newyearday(d):
    if d.month == 1 and d.day == 1:
        IsNewYearDay = '1' # 元旦节
    elif d.month == 12 and d.is_month_end == True:
        IsNewYearDay = '1'  # 元旦节前一天
    elif d.month == 1 and d.day <= 3:
        IsNewYearDay = '1'  # 元旦节后两天
    else:
        IsNewYearDay = '0'  # 非元旦节
    return IsNewYearDay

#### 1.5 获取起飞时间为月初、月中或者月末字段

In [69]:
def get_month_segment(d):
    if d.day <= 10:
        MonthSegment = '1' # 月初
    elif d.day <= 20:
        MonthSegment = '2' # 月中
    else:
        MonthSegment = '3' # 月末
    return MonthSegment

#### 1.6 获取起飞时间是否为节假日

In [70]:
def is_festival(d):
    m = d.month # 获取月份
    if m <= 2:                      # 春节
        IsFestival = '1'
    elif m >= 7 and m <= 9:         # 暑假
        IsFestival = '1'
    elif is_labourday(d) == '1':    # 劳动节
        IsFestival = '1'
    elif is_nationalday(d) == '1': # 国庆节
        IsFestival = '1'
    elif is_newyearday(d) == '1':  # 元旦节
        IsFestival = '1'
    else:
        IsFestival = '0'            # 非节假日
    return IsFestival

### 2. 缺失值填充函数

In [73]:
def data_missing_process(df,date_range,features):
    cols = ['depDate','orderDate']
    cols.extend(features)
    fill_data = pd.DataFrame(columns = cols) # 需要填充的数据
    del_data = []                            # 需要删除的数据
    i = 0 # 行号
    start_time = time.time()
    for dep_date in date_range:
        res = []
        s = (dep_date-datetime.timedelta(59)).strftime('%Y-%m-%d') # 起飞前60天的搜索日期转为字符
        t_series = pd.DataFrame(pd.date_range(s,dep_date),columns=['orderDate']) # 生成需要的起飞前60天的搜索日期序列
        t_series_true = pd.DataFrame(df['orderDate'][df['depDate']==dep_date],columns=['orderDate']) # 实际起飞日期的搜索日期序列
        res = pd.concat([t_series,t_series_true]) #先添加进来
        res = res.drop_duplicates(subset=['orderDate'],keep=False) #去重，得到起飞日期的缺失搜索日期序列
        if not res.empty and len(res) <= 15 : # 数据填充，缺失超过15天不填充
            for order_date in res['orderDate']:
                fill_data.loc[i,'depDate'] = dep_date      # 缺失数据的起飞日期
                fill_data.loc[i,'orderDate'] = order_date  # 缺失数据的搜索日期
                order_last,order_next = order_date,order_date
                while 1: # 找到缺失搜索日期非null的最近前一天，若搜索日期超过起飞日期前60天，则用后一天数据填充
                    order_last-=datetime.timedelta(1)
                    fill_last = df.loc[df['depDate'] == dep_date,features]
                    fill_last = fill_last.loc[df['orderDate'] == order_last]
                    if not fill_last.empty or order_last< (dep_date-datetime.timedelta(59)):
                        break
                while 1: # 找到搜索日期非null的最近后一天，若搜索日期超过起飞日期，则用前一天数据填充
                    order_next+=datetime.timedelta(1)
                    fill_next = df.loc[df['depDate'] == dep_date,features]
                    fill_next = fill_next.loc[df['orderDate'] == order_next] 
                    if not fill_next.empty or order_next>dep_date:
                        break
    #             填充剩余所有特征
                if order_next>dep_date and not fill_last.empty:                             # 后过界，用前一天数据填充
                    fill_data.iloc[i,2:] = fill_last.values
                elif order_last< (dep_date-datetime.timedelta(59)) and not fill_next.empty: # 前过界，用后一天数据填充
                    fill_data.iloc[i,2:] = fill_next.values
                elif fill_last.empty and fill_next.empty:                                   # 前后都过界，暂时用0填充
                    fill_data.iloc[i,2:] = 0
                else:                                                                       # 前后两天取均值填充
                    fill_data.iloc[i,2:] = ((fill_last.values+fill_next.values)/2)
                i+=1
        if len(res)>15:
            del_data.append(dep_date)
        print("缺失数据为%s"%res)
        print('%s缺失填充完成！缺失数据大小为%s'%(dep_date,len(res)))
    print('需删除的数据',del_data)
    cost_time = time.time()-start_time
    print('缺失数据填充耗时：',cost_time)
    return fill_data,del_data